<a href="https://colab.research.google.com/github/stixmal/praktikum_project_ds/blob/main/%D0%9E%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%20%D1%82%D0%BE%D0%BA%D1%81%D0%B8%D1%87%D0%BD%D1%8B%D1%85%20%D0%BA%D0%BE%D0%BC%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%80%D0%B8%D0%B5%D0%B2/toxic_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построили модель со значением метрики качества *F1* не меньше 0.75. 


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *`text`* - в нём содержит текст комментария, а *`toxic`* — целевой признак.
***

## 1. Подготовка данных

In [ ]:
# импорт библиотеки pandas и numpy
import pandas as pd 
import numpy as np  

# импорт модуля display
from IPython.display import display  

# отключение предупреждений
import warnings
warnings.filterwarnings('ignore') 

# импорт лемматизатора
import re
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

# импорт модуля разделения на выборки
from sklearn.model_selection import train_test_split

# счётчик величин TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# инструментарий естественного языка - стоп-слова
from nltk.corpus import stopwords

# импорт модели логистической регрессии
from sklearn.linear_model import LogisticRegression

# импорт модели случайного леса
from sklearn.ensemble import RandomForestClassifier

# импорт модели градиентного бустинга
import lightgbm as lgb

# импорт метрик
from sklearn.metrics import f1_score

In [ ]:
# чтение файла с данными
df = pd.read_csv('/datasets/toxic_comments.csv')

# визуальный осмотр  
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [ ]:
# функция для лемматизации текста
lem = WordNetLemmatizer()
def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lem.lemmatize(w) for w in word_list])
        
    return lemm_text

In [ ]:
# функция для очистки текста
def clear_text(text):
    eng_text = re.sub(r'[^a-zA-Z ]', ' ', text)
    clear_text = " ".join(eng_text.split())
    
    return clear_text

In [ ]:
# лемматизированный текст
df['lemm_text'] = df['text'].apply(lambda i: lemmatize(clear_text(i)))

In [ ]:
df

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,And for the second time of asking when your vi...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That is a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm there no actual article for prosti...
159569,And it looks like it was actually you who put ...,0,And it look like it wa actually you who put on...


Разделили на признаки.

In [ ]:
X = df['lemm_text']
y = df['toxic']

Разделили на выборки train и test.

In [ ]:
# раздел на 75 % обучающей и 25 % тестовой выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

Проконтролировали размер выборок.

In [ ]:
X_train.shape, y_train.shape

((119678,), (119678,))

In [ ]:
X_test.shape, y_test.shape

((39893,), (39893,))

***

Вычислили TF-IDF для корпуса текстов.

In [ ]:
corpus_train = X_train.values.astype('U')
corpus_test = X_test.values.astype('U')

Cоздали счётчик, указав в нём стоп-слова.

In [ ]:
stopwords = stopwords.words('english')

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

Чтобы посчитать TF-IDF для корпуса текстов, вызвали функцию fit_transform().

In [ ]:
tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test)

In [ ]:
# признаки и ответы
y_train = y_train.values
X_train = tf_idf_train

y_test = y_test.values
X_test = tf_idf_test

## 2. Обучение

### 2.1 Логистическая регрессия

In [ ]:
log_reg = LogisticRegression(class_weight='balanced', n_jobs=-1, random_state=12345)

In [ ]:
# обучение и предсказания модели
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)

In [ ]:
f1_score(y_test, y_pred)

0.7503759398496241

### 2.2 Случайный лес

In [ ]:
rfc = RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=12345)

In [ ]:
# обучение и предсказания модели
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [ ]:
f1_score(y_test, y_pred)

0.6025956957450304

### 2.3 Градиентный бустинг

In [ ]:
gbm = lgb.LGBMClassifier(learning_rate=0.1, num_leaves=10, n_estimators=40, class_weight='balanced', random_state=12345)

In [ ]:
# обучение и предсказания модели
gbm.fit(X_train, y_train)
y_pred = gbm.predict(X_test)

In [ ]:
f1_score(y_test, y_pred)

0.6701557632398755

## 3. Выводы

Из всех обученных моделей, как ни странно, лучшие результаты показала логистическая регрессия с результотом f1_score = 0.75. Тогда как решения от разных методов над деревьями не дотягивают до необходимого минимума.